In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def initialize_lattice(size):
    return np.ones((size, size), dtype=int)

In [ ]:
def compute_magnetization(lattice):
    size = lattice.shape[0]
    return np.sum(lattice)/(size**2)

In [ ]:
def metropolis_step(lattice, beta):
    size = lattice.shape[0]
    i, j = np.random.randint(0, size, 2)

    neighbor_sum = lattice[(i + 1) % size, j] + lattice[(i - 1) % size, j] + lattice[i, (j + 1) % size] + lattice[i, (j - 1) % size]

    delta_energy = 2 * lattice[i, j] * neighbor_sum
    if delta_energy < 0 or np.random.rand() < np.exp(-beta * delta_energy):
        lattice[i, j] *= -1

In [ ]:
def metropolis_simulation(size, beta, kmax):
    lattice = initialize_lattice(size)
    running_mean = compute_magnetization(lattice)
    running_var = 0

    for k in range(1, kmax + 1):
        metropolis_step(lattice, beta)
        magnetization = compute_magnetization(lattice)
        
        running_mean_prev = running_mean
        running_mean = running_mean + (magnetization-running_mean) / (k+1)
        if k >= 2:
            running_var = ((k-2)*running_var + (magnetization - running_mean)**2) / (k-1)

    return running_mean, running_var

In [ ]:
beta_values = np.arange(0.2, 1.01, 0.01)
kmax = 10**7
lattice_size = 30
mean_arr = np.array([])
std_arr = np.array([])

for beta in beta_values:
    mean_magnetizations, var = metropolis_simulation(lattice_size, beta, kmax)
    mean_arr = np.append(mean_arr, mean_magnetizations)
    std_arr = np.append(std_arr, np.sqrt(var))
    plt.plot(beta, mean_magnetizations, 'ro', markersize=2)

plt.plot(beta_values, mean_arr + std_arr, 'r--', markersize=2)
plt.plot(beta_values, mean_arr - std_arr, 'r--', markersize=2)
    
condition = [beta_values < 0.4408, beta_values >= 0.4408]
uB = np.piecewise(beta_values, condition, [0, lambda beta: (1 - (np.sinh(2*beta))**(-4))**(1/8)])
plt.plot(beta_values, uB, 'blue', markersize=2)
    
plt.grid()
plt.xlabel('Beta (β)')
plt.ylabel('Mean Magnetization')
plt.title('Mean Magnetization vs Beta for 2D Ising Model')
plt.show()